In [9]:
import os
import sys
#sys.path.append("/workspace")
import numpy as np
import tensorflow as tf
from src.data.general_processor import Utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle
from sklearn.preprocessing import minmax_scale
import pathlib
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


from src.models.model_architectures.model_1DCNN import HopefullNet
from src.data.make_dataset import BCIDataset

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
root_path = pathlib.Path().resolve().parents[0]

# DATA
data_root = "C:/Users/annag/OneDrive - Danmarks Tekniske Universitet/Semester_04/Special_Course_BCI/03_code/BCI_stroke_rehab/data/raw/"

# MODEL
save_path = os.path.join(root_path, "models/1D_CNN/ours_6_pairs/fine_tuned/physionet_base/")

## Device settings
tf.autograph.set_verbosity(0)
physical_devices = tf.config.experimental.list_physical_devices('CPU')

#Params    
channels = [["C3","C4"],["F3","F4"],["P3","P4"],["FP1","FP2"],["F7","F8"],["T3","T4"]]
two_class = True

if two_class:
    input_shape = (None, 500, 2)
    loss = tf.keras.losses.binary_crossentropy
else:
    input_shape = (None, 640, 2)
    loss = tf.keras.losses.categorical_crossentropy

In [11]:
# Test subjects for testing transfer learning
test_subjects = ["dani", "ivo", "pablo", "huiyu", "manu", "fabio", "anna", "luisa", "sarah", "irene", "jan"]
#test_subjects = ["fabio"]

measurements = []

# Get dataset
trainingset = BCIDataset(data_root, test_subjects, [], measurement_length=4)
trainingset.validate_data()
trainingset.apply_bandpass_filtering(selected_data="sample")

Subjects: ['dani', 'ivo', 'pablo', 'huiyu', 'manu', 'fabio', 'anna', 'luisa', 'sarah', 'irene', 'jan']
Total invalid samples: 63


In [21]:
log_accuracies = {}
for subj in test_subjects: 
    # Modelname
    modelname = "finetuned_sub_"+subj+"_2023-04-27_22-48-56.h5"

    log_accuracies[subj] = {}
    # Load data
    x, y, ch_pairs = trainingset.load_subject_data(subj, channels)
    indexes = range(0,len(x))

    # Reshape for scaling
    reshaped_x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    
#     train_idx, test_idx, y_train_raw, y_test = train_test_split(indexes,
#                                                                 y,
#                                                                 stratify=y,
#                                                                 test_size=0.20,
#                                                                 random_state=42)
    # Get train-/test- indices of 5 fold Cross Validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for i, (train_index, test_index) in enumerate(skf.split(reshaped_x, y)):
        # Get model path
        model_path = os.path.join(save_path+"sub_"+subj+"/"+"fold_"+str(i)+"/", modelname)
    
        print(f"Fold {i}:")
        #print(f"  Train: index={train_index}")
        #print(f"  Test:  index={test_index}")

        # x_train_raw = reshaped_x[train_index]
        x_test_raw = reshaped_x[test_index]
        # y_train_raw = y[train_index]
        y_test = y[test_index]
        ch_pairs_test = ch_pairs[test_index]
        
        # Normalize array
        reshaped_x_scaled = minmax_scale(x_test_raw, axis=1)
        # x_test = reshaped_x_scaled.reshape(reshaped_x_scaled.shape[0], int(reshaped_x_scaled.shape[1]/2),2).astype(np.float64)
        x_test = reshaped_x_scaled.reshape(reshaped_x_scaled.shape[0], x.shape[1], x.shape[2]).astype(np.float64)
        x_test = np.swapaxes(x_test,1,2)
    
        # Transform labels in int-values (0/1)
        y_test_01 = []
        for y_label in y_test:
            if y_label == 'L':
                y_test_01.append(0)
            elif y_label == 'R':
                y_test_01.append(1)
            else:
                print("Test Labels are different than L or R...")

        y_test = np.array(y_test_01)

        ## Load model   
        model = HopefullNet(inp_shape = (input_shape[1],input_shape[2]), two_class=two_class)
        model.build(input_shape)
        model.load_weights(model_path)

        learning_rate = 1e-4
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

        testLoss, testAcc = model.evaluate(x_test, y_test)
        print('\nAccuracy:', testAcc)
        print('\nLoss: ', testLoss)

        ## Test model with subject data 
        for ch_pair in channels:
            ch_pair_name = ' '.join(ch_pair)
            print(f"Test model with subject {subj}: and channel pair: {ch_pair}\n")
            
            try:
                len(log_accuracies[subj][ch_pair_name])
            except:
                log_accuracies[subj][ch_pair_name] = []
            

            # Get samples of channel pair
            idx = np.where(ch_pairs_test == ch_pair)[0]
            x_test_ch = x_test[idx]
            y_test_ch = y_test[idx]

            yPred = model.predict(x_test_ch)

            # convert label in string
            if two_class:
                yTestClass = []
                for label in y_test_ch:
                    if label==0:
                        yTestClass.append("L")
                    elif label==1:
                        yTestClass.append("R")
                    else:
                        print("Label not found.")

                yPredClass = []
                for label in yPred:
                    if label<0.5:
                        yPredClass.append("L")
                    elif label>=0.5:
                        yPredClass.append("R")
                    else:
                        print("Label not found.")

                target_names = ["L", "R"]
            else:  
                yTestClass = np.argmax(y_test_ch, axis=1)
                yPredClass = np.argmax(yPred,axis=1)
                target_names=["B", "R", "RL", "L", "F"]
                
            # Calc accuracy
            testAcc_ch = accuracy_score(yTestClass, yPredClass)
            log_accuracies[subj][ch_pair_name].append(testAcc_ch)
            
            

#             print('\n Classification report \n\n',
#               classification_report(
#                   yTestClass,
#                   yPredClass,
#                    target_names=target_names
#               )
#             )

#             print('\n Confusion matrix \n\n',
#               confusion_matrix(
#                   yTestClass,
#                   yPredClass,
#               )
#             )

Get data from subjects: ['dani']
Fold 0:
Predicting 2 classes.
5/5 [==============================] - 1s 38ms/step - loss: 0.0563 - accuracy: 0.9792

Accuracy: 0.9791666865348816

Loss:  0.05631283298134804
Test model with subject dani: and channel pair: ['C3', 'C4']

2/2 [==============================] - 0s 57ms/step
Test model with subject dani: and channel pair: ['F3', 'F4']

2/2 [==============================] - 0s 17ms/step
Test model with subject dani: and channel pair: ['P3', 'P4']

2/2 [==============================] - 0s 55ms/step
Test model with subject dani: and channel pair: ['FP1', 'FP2']

2/2 [==============================] - 0s 36ms/step
Test model with subject dani: and channel pair: ['F7', 'F8']

2/2 [==============================] - 0s 26ms/step
Test model with subject dani: and channel pair: ['T3', 'T4']

2/2 [==============================] - 0s 46ms/step
Fold 1:
Predicting 2 classes.
5/5 [==============================] - 2s 38ms/step - loss: 0.1070 - accuracy

2/2 [==============================] - 0s 20ms/step
Test model with subject ivo: and channel pair: ['FP1', 'FP2']

2/2 [==============================] - 0s 29ms/step
Test model with subject ivo: and channel pair: ['F7', 'F8']

2/2 [==============================] - 0s 24ms/step
Test model with subject ivo: and channel pair: ['T3', 'T4']

2/2 [==============================] - 0s 47ms/step
Get data from subjects: ['pablo']
Fold 0:
Predicting 2 classes.
5/5 [==============================] - 1s 37ms/step - loss: 0.0402 - accuracy: 0.9716

Accuracy: 0.9716312289237976

Loss:  0.04021410644054413
Test model with subject pablo: and channel pair: ['C3', 'C4']

2/2 [==============================] - 0s 34ms/step
Test model with subject pablo: and channel pair: ['F3', 'F4']

2/2 [==============================] - 0s 14ms/step
Test model with subject pablo: and channel pair: ['P3', 'P4']

2/2 [==============================] - 0s 49ms/step
Test model with subject pablo: and channel pair: ['FP1

1/1 [==============================] - 0s 94ms/step
Fold 4:
Predicting 2 classes.
4/4 [==============================] - 1s 40ms/step - loss: 0.1227 - accuracy: 0.9688

Accuracy: 0.96875

Loss:  0.12271969020366669
Test model with subject huiyu: and channel pair: ['C3', 'C4']

2/2 [==============================] - 0s 42ms/step
Test model with subject huiyu: and channel pair: ['F3', 'F4']

2/2 [==============================] - 0s 17ms/step
Test model with subject huiyu: and channel pair: ['P3', 'P4']

1/1 [==============================] - 0s 99ms/step
Test model with subject huiyu: and channel pair: ['FP1', 'FP2']

2/2 [==============================] - 0s 17ms/step
Test model with subject huiyu: and channel pair: ['F7', 'F8']

2/2 [==============================] - 0s 33ms/step
Test model with subject huiyu: and channel pair: ['T3', 'T4']

2/2 [==============================] - 0s 19ms/step
Get data from subjects: ['manu']
Fold 0:
Predicting 2 classes.
4/4 [=========================

2/2 [==============================] - 0s 45ms/step
Test model with subject fabio: and channel pair: ['P3', 'P4']

2/2 [==============================] - 0s 30ms/step
Test model with subject fabio: and channel pair: ['FP1', 'FP2']

2/2 [==============================] - 0s 25ms/step
Test model with subject fabio: and channel pair: ['F7', 'F8']

2/2 [==============================] - 0s 50ms/step
Test model with subject fabio: and channel pair: ['T3', 'T4']

2/2 [==============================] - 0s 34ms/step
Fold 4:
Predicting 2 classes.
5/5 [==============================] - 1s 32ms/step - loss: 0.0594 - accuracy: 0.9792

Accuracy: 0.9791666865348816

Loss:  0.05940830707550049
Test model with subject fabio: and channel pair: ['C3', 'C4']

2/2 [==============================] - 0s 48ms/step
Test model with subject fabio: and channel pair: ['F3', 'F4']

2/2 [==============================] - 0s 33ms/step
Test model with subject fabio: and channel pair: ['P3', 'P4']

2/2 [==============


2/2 [==============================] - 0s 19ms/step
Fold 3:
Predicting 2 classes.
5/5 [==============================] - 1s 34ms/step - loss: 0.0025 - accuracy: 1.0000

Accuracy: 1.0

Loss:  0.002547824988141656
Test model with subject luisa: and channel pair: ['C3', 'C4']

2/2 [==============================] - 0s 12ms/step
Test model with subject luisa: and channel pair: ['F3', 'F4']

2/2 [==============================] - 0s 19ms/step
Test model with subject luisa: and channel pair: ['P3', 'P4']

2/2 [==============================] - 0s 37ms/step
Test model with subject luisa: and channel pair: ['FP1', 'FP2']

2/2 [==============================] - 0s 20ms/step
Test model with subject luisa: and channel pair: ['F7', 'F8']

2/2 [==============================] - 0s 47ms/step
Test model with subject luisa: and channel pair: ['T3', 'T4']

2/2 [==============================] - 0s 10ms/step
Fold 4:
Predicting 2 classes.
5/5 [==============================] - 1s 23ms/step - loss: 2.447

2/2 [==============================] - 0s 23ms/step
Test model with subject irene: and channel pair: ['P3', 'P4']

2/2 [==============================] - 0s 40ms/step
Test model with subject irene: and channel pair: ['FP1', 'FP2']

2/2 [==============================] - 0s 25ms/step
Test model with subject irene: and channel pair: ['F7', 'F8']

2/2 [==============================] - 0s 27ms/step
Test model with subject irene: and channel pair: ['T3', 'T4']

2/2 [==============================] - 0s 34ms/step
Fold 3:
Predicting 2 classes.
4/4 [==============================] - 1s 41ms/step - loss: 0.0768 - accuracy: 0.9587

Accuracy: 0.9586777091026306

Loss:  0.07677394896745682
Test model with subject irene: and channel pair: ['C3', 'C4']

2/2 [==============================] - 0s 49ms/step
Test model with subject irene: and channel pair: ['F3', 'F4']

2/2 [==============================] - 0s 29ms/step
Test model with subject irene: and channel pair: ['P3', 'P4']

2/2 [==============

In [26]:
# Calc mean accuracies
for sub in test_subjects:
    print(f"Subject {sub}:\n")
    for ch_pair in channels:
        ch_pair_name = ' '.join(ch_pair)
        mean_acc = np.mean(log_accuracies[sub][ch_pair_name])
        print(f"Mean acc {ch_pair_name} = {mean_acc}")
    print("\n")

Subject dani:

Mean acc C3 C4 = 0.99
Mean acc F3 F4 = 0.9563892339544513
Mean acc P3 P4 = 0.9286153846153846
Mean acc FP1 FP2 = 0.993103448275862
Mean acc F7 F8 = 0.9894736842105264
Mean acc T3 T4 = 0.9843076923076923


Subject ivo:

Mean acc C3 C4 = 0.99
Mean acc F3 F4 = 0.9388944099378882
Mean acc P3 P4 = 0.9160012740882305
Mean acc FP1 FP2 = 0.980952380952381
Mean acc F7 F8 = 0.992
Mean acc T3 T4 = 0.9846153846153847


Subject pablo:

Mean acc C3 C4 = 1.0
Mean acc F3 F4 = 0.9488215488215488
Mean acc P3 P4 = 0.9764895330112722
Mean acc FP1 FP2 = 1.0
Mean acc F7 F8 = 1.0
Mean acc T3 T4 = 1.0


Subject huiyu:

Mean acc C3 C4 = 1.0
Mean acc F3 F4 = 0.913913043478261
Mean acc P3 P4 = 0.9733706246292059
Mean acc FP1 FP2 = 1.0
Mean acc F7 F8 = 1.0
Mean acc T3 T4 = 0.9913043478260869


Subject manu:

Mean acc C3 C4 = 1.0
Mean acc F3 F4 = 0.8916452293836195
Mean acc P3 P4 = 0.982
Mean acc FP1 FP2 = 1.0
Mean acc F7 F8 = 1.0
Mean acc T3 T4 = 0.990909090909091


Subject fabio:

Mean acc C3 C4 =